# LSTM for Spoken Digit Classification

This notebook builds and trains a recurrent neural network (LSTM) to classify spoken digits (0–9) from audio recordings.

- Dataset: [Free Spoken Digit Dataset (FSDD)](https://github.com/Jakobovski/free-spoken-digit-dataset)
- Framework: PyTorch
- Architecture: RNN with LSTM layers

In [1]:
import sys
import os

# Add the project root (parent of current folder) to Python path
project_root_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root_dir)


## Load Model Configuration from YAML

To make the training pipeline configurable and modular, we store model parameters like number of LSTM layers, hidden size, and learning rate etc in a YAML file. This structure enables quick adaptation to related tasks B, and C.

This section loads the model configuration using a custom utility function.

In [2]:
import src.utils as utils

In [3]:
utils.set_seed(42)

[INFO] Random seed set to: 42


In [4]:
import yaml
import json

model_config_path = os.path.join(project_root_dir, 'config', 'model_config.yaml')
model_config = utils.read_yaml_file(model_config_path)
# print(json.dumps(model_config, indent=2))

## Load and Split Dataset for Training and Evaluation

In this section, we load the recordings data from disk, generate data-label pairs, and split them into training and test sets according to the `test_size` defined in the YAML file.

Using `test_size` and `seed` from the YAML config ensures that experiments are reproducible and easily tunable for other tasks by simply updating the configuration.


In [5]:
data_path = model_config['dataset']['path']
test_data_size = model_config['data_splitting']['test_size']
seed = model_config['experiment']['seed']

In [6]:
data_label_pairs = utils.prepare_data_label_pairs(data_path)

In [7]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data_label_pairs, test_size=test_data_size, random_state=seed)

## Transform Raw Data into PyTorch Dataset Objects

The `AudioFeaturesDataset` class converts raw data-label pairs into PyTorch-compatible datasets that provide easy access to samples and labels.

AudioFeaturesDataset is a custom dataset class that:

- Loads audio recordings of spoken digits along with their labels.
- Optionally cleans the audio by filtering out noise.
- Extracts MFCC features (a common speech feature).
- Pads or trims these features to a fixed length so all inputs have the same shape.
- Works with PyTorch to provide samples one-by-one when training or testing a model.
- It helps prepare your audio data in the right format for training neural networks efficiently.


In [8]:
from src.data_preprocessor import AudioFeaturesDataset

train_dataset = AudioFeaturesDataset(train_data)
test_dataset = AudioFeaturesDataset(test_data)

In [9]:
print(f"Train size: {len(train_dataset)}")
print(f"Test size: {len(test_dataset)}")

Train size: 2400
Test size: 600


## Create DataLoaders for Batch Processing

Using PyTorch DataLoaders, we enable efficient loading, batching, and shuffling of data during training and evaluation.

In [13]:
import torch

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [14]:
input_dim = model_config['model']['input_dim']
hidden_dim = model_config['model']['hidden_dim']
num_layers = model_config['model']['num_layers']
output_dim = model_config['model']['output_dim']

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Static Quantization

In [17]:
import src.model as model

hidden_dim = 24
quant_model = model.LSTMClassifier(input_dim=input_dim,
                       hidden_dim=hidden_dim,
                       num_layers=num_layers,
                       output_dim=output_dim).to(device)

In [18]:
memory_constraint_model_load_path = os.path.join(project_root_dir, 'outputs', 'models', 'task-b-part-1_weights.pth')
quant_model.load_state_dict(torch.load(memory_constraint_model_load_path))

<All keys matched successfully>

In [19]:
quant_model.eval()

LSTMClassifier(
  (lstm): LSTM(13, 24, num_layers=2, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=24, out_features=10, bias=True)
)

In [20]:
import src.quantize as quantize
static_quant_model = quantize.StaticQuantizableModel(quant_model)

In [23]:
static_quant_model.qconfig = torch.quantization.get_default_qconfig('x86')  # For edge devices
# torch.backends.quantized.engine = 'fbgemm'

static_quant_model_prepared = torch.quantization.prepare(static_quant_model)

In [24]:
calibration_indices = list(range(100))
calibration_dataset = torch.utils.data.Subset(test_dataset, calibration_indices)

calibration_dataloader = torch.utils.data.DataLoader(calibration_dataset, batch_size=32)

In [25]:
static_quant_model_prepared_cpu = static_quant_model_prepared.to('cpu')
device = torch.device("cpu")

quantize.calibrate(static_quant_model_prepared, calibration_dataloader, device)

In [26]:
static_quantized_model_converted = torch.quantization.convert(static_quant_model_prepared)

In [28]:
from src.evaluate import ModelEvaluator
print(device)
quant_test_instance = ModelEvaluator(
    static_quantized_model_converted, 
    test_loader,
    device
)

cpu


In [29]:
quant_test_instance.evaluate()

/home/pavan/Music/spectrum/spectgramer/lib/python3.10/site-packages/torch/ao/nn/quantizable/modules/rnn.py:543: UserWarning: All inputs of this cat operator must share the same quantization parameters. Otherwise large numerical inaccuracies may occur. (Triggered internally at /pytorch/aten/src/ATen/native/quantized/cpu/TensorShape.cpp:167.)
  cx_tensor = torch.stack(cx_list)



 Accuracy on test data: 76.00%

 Classification Report:
              precision    recall  f1-score   support

           0     0.5823    0.8846    0.7023        52
           1     0.8936    0.6462    0.7500        65
           2     0.7234    0.5397    0.6182        63
           3     0.5941    1.0000    0.7453        60
           4     0.9636    0.9636    0.9636        55
           5     0.7500    0.5660    0.6452        53
           6     0.9149    0.6615    0.7679        65
           7     0.7797    0.7797    0.7797        59
           8     0.8936    0.6885    0.7778        61
           9     0.7692    0.8955    0.8276        67

    accuracy                         0.7600       600
   macro avg     0.7864    0.7625    0.7577       600
weighted avg     0.7897    0.7600    0.7582       600


 Confusion Matrix:
[[46  0  2  3  0  0  0  1  0  0]
 [ 2 42  3  1  2  2  0 12  0  1]
 [25  0 34  4  0  0  0  0  0  0]
 [ 0  0  0 60  0  0  0  0  0  0]
 [ 0  2  0  0 53  0  0  0  0  0]

In [30]:
for name, module in static_quantized_model_converted.named_modules():
    if isinstance(module, torch.nn.quantized.Linear):
        print(f"\n{name} is a quantized Linear layer")
        weight = module.weight()
        print("Weight type:", type(weight))
        print("Weight dtype:", weight.dtype)
        print("Weight shape:", weight.shape)
        print("---")


model.lstm.layers.0.layer_fw.cell.igates is a quantized Linear layer
Weight type: <class 'torch.Tensor'>
Weight dtype: torch.qint8
Weight shape: torch.Size([96, 13])
---

model.lstm.layers.0.layer_fw.cell.hgates is a quantized Linear layer
Weight type: <class 'torch.Tensor'>
Weight dtype: torch.qint8
Weight shape: torch.Size([96, 24])
---

model.lstm.layers.1.layer_fw.cell.igates is a quantized Linear layer
Weight type: <class 'torch.Tensor'>
Weight dtype: torch.qint8
Weight shape: torch.Size([96, 24])
---

model.lstm.layers.1.layer_fw.cell.hgates is a quantized Linear layer
Weight type: <class 'torch.Tensor'>
Weight dtype: torch.qint8
Weight shape: torch.Size([96, 24])
---

model.fc is a quantized Linear layer
Weight type: <class 'torch.Tensor'>
Weight dtype: torch.qint8
Weight shape: torch.Size([10, 24])
---


In [33]:
total_fp32_bytes = 0
total_int8_bytes = 0

for name, module in static_quantized_model_converted.named_modules():
    if isinstance(module, torch.nn.quantized.Linear):
        weight = module.weight()
        if weight.is_quantized:
            num_elements = weight.numel()
            total_fp32_bytes += num_elements * 4  # FP32
            total_int8_bytes += num_elements * 1  # INT8

print(f"Estimated FP32 size: {total_fp32_bytes / 1024:.2f} KB")
print(f"Estimated INT8 size: {total_int8_bytes / 1024:.2f} KB")
print(f"Compression ratio: {total_fp32_bytes / total_int8_bytes:.2f}x")

Estimated FP32 size: 32.81 KB
Estimated INT8 size: 8.20 KB
Compression ratio: 4.00x


## Task B — Retrain Under Memory Constraints

- All parameters of any one layer must fit into memory simultaneously
- Maximum memory available for layer parameters is 36 KB

Since Pytorch stores all the layer parameters as floating point values, as 32-bit floats(4 bytes per parameter), this implies that the 
maximum number of parameters should not exceed

$$
\text{Max total number of parameters} = \frac{36\,\text{KB}}{4\,\text{bytes}} = \frac{36 \times 1024}{4} = 9,216 \text{ parameters}
$$

## Model Parameter Breakdown for 2 Layers LSTM
The following calculations are based on the parameter definitions from PyTorch's LSTM implementation, as described in the \href{https://docs.pytorch.org/docs/stable/generated/torch.nn.LSTM.html\#torch.nn.LSTM}{official documentation}. 

Let’s define the following variables:

$$
\begin{aligned}
\text{Input dimension} &= i \\
\text{Hidden dimension} &= h \\
\text{Output dimension} &= o \\
\text{Number of LSTM layers} &= 2 \\
\text{Fully Connected (Linear) layer count} &= 1 \\
\end{aligned}
$$

These will be used to calculate the total number of parameters in the model.

---

## LSTM Layer Parameters

Each LSTM layer has 4 internal gates (input, forget, cell, output).  
So each layer has:

$$
\begin{aligned}
W\_{ih} &: \text{Weights from input to hidden} \rightarrow \text{shape: } (4 \times h, i) \\
W\_{hh} &: \text{Weights from hidden to hidden} \rightarrow \text{shape: } (4 \times h, h) \\
b\_{ih}, b\_{hh} &: \text{Biases for each gate} \rightarrow \text{shape: } (4 \times h)
\end{aligned}
$$

### Calculations:

$$
\begin{aligned}
&\textbf{Layer 1 Parameters} \\
&W\_ih: 4 \times h \times i \\
&W\_hh: 4 \times h \times h \\
&b\_ih: 4 \times h \\
&b\_hh: 4 \times h \\
&\\
&\textbf{Layer 2 Parameters} \\
&W\_ih: 4 \times h \times h \\
&W\_hh: 4 \times h \times h \\
&b\_ih: 4 \times h \\
&b\_hh: 4 \times h \\
&\\
&\text{Total LSTM parameters} = (4 \times h \times i) + (4 \times h \times h) + (8 \times h) = 4hi + 4h^2 + 8h
\end{aligned}
$$

---

## Fully Connected (Linear) Layer

- Input features = $$\text{hidden\_dim} = h$$  
- Output features = $$\text{output\_dim} = o$$

### Calculations:
$$
\begin{aligned}
&\text{Weights}: \quad h \times o \\
&\text{Bias}: \quad o \\
&\textbf{Total Linear parameters} = h \cdot o + o = o(h + 1)
\end{aligned}
$$

---

## Total Parameters

The total number of parameters in the model is the sum of the LSTM and Linear layer parameters:

$$
\text{Total Parameters} = \text{Total LSTM Parameters} + \text{Total Linear Parameters}
$$

From earlier calculations:

- $\text{Total LSTM Parameters} = 4hi + 4h^2 + 8h$
- $\text{Total Linear Parameters} = h \cdot o + o = o(h + 1)$

Therefore:

$$
\text{Total Parameters} = (4hi + 4h^2 + 8h) + o(h + 1)
$$

Or more compactly:

$$
\boxed{\text{Total Parameters} = 4hi + 4h^2 + 8h + o(h + 1)}
$$

---


## Designing a 2-Layer LSTM Under a 36 KB Memory Constraint

Given an output dimension of $10$ (representing 10 classes or digits) and an input dimension of $13$ (corresponding to 13 MFCC coefficients per time step), the total number of parameters in the model reduces to the following quadratic expression:

$$
\text{Total Parameters} = 12h^2 + 78h + 10
$$

Here, $ h $ (the hidden dimension) remains the only variable we need to solve for.

Since the memory constraint allows for a maximum of \textbf{9,216 parameters}, the hidden dimension must satisfy:

$$
12h^2 + 78h + 10 \leq 9216
$$

The maximum valid integer value of $ h $ that satisfies the inequality is:
$$
h = 24
$$

---

## LSTM Model Definition

A simple `n`-layer LSTM followed by a fully connected output layer. Variable `n` is defined in the configuration YAML file

In [13]:
import src.model as model_with_constraints

hidden_dim = 24
memory_constraint_model = model_with_constraints.LSTMClassifier(input_dim=input_dim,
                                           hidden_dim=hidden_dim,
                                           num_layers=num_layers,
                                           output_dim=output_dim).to(device)

## Training Loop

In [14]:
learning_rate = model_config['training']['learning_rate']
epochs = model_config['training']['epochs']

In [15]:
from src.train import ModelTrainer
trainer_instance_2 = ModelTrainer(
    memory_constraint_model, 
    epochs,
    train_loader,
    device,
    learning_rate
)

In [16]:
trainer_instance_2.train()

Epoch [1/20], Loss: 171.4650, Accuracy: 29.38%
Epoch [2/20], Loss: 138.1799, Accuracy: 49.17%
Epoch [3/20], Loss: 99.5578, Accuracy: 57.54%
Epoch [4/20], Loss: 84.4610, Accuracy: 65.42%
Epoch [5/20], Loss: 73.8342, Accuracy: 70.62%
Epoch [6/20], Loss: 65.2555, Accuracy: 73.54%
Epoch [7/20], Loss: 57.3031, Accuracy: 76.33%
Epoch [8/20], Loss: 52.0835, Accuracy: 82.04%
Epoch [9/20], Loss: 44.7541, Accuracy: 81.75%
Epoch [10/20], Loss: 39.6640, Accuracy: 85.25%
Epoch [11/20], Loss: 38.6156, Accuracy: 85.62%
Epoch [12/20], Loss: 32.0929, Accuracy: 88.58%
Epoch [13/20], Loss: 28.5191, Accuracy: 90.75%
Epoch [14/20], Loss: 28.1301, Accuracy: 89.33%
Epoch [15/20], Loss: 28.3795, Accuracy: 90.38%
Epoch [16/20], Loss: 27.9978, Accuracy: 91.21%
Epoch [17/20], Loss: 24.3191, Accuracy: 94.29%
Epoch [18/20], Loss: 21.8360, Accuracy: 93.58%
Epoch [19/20], Loss: 22.1747, Accuracy: 92.79%
Epoch [20/20], Loss: 22.3266, Accuracy: 94.96%


In [17]:
_, _ = utils.get_model_params_size(memory_constraint_model)


Layer-wise parameter counts:
lstm.weight_ih_l0              -> 1,248 params
lstm.weight_hh_l0              -> 2,304 params
lstm.bias_ih_l0                -> 96 params
lstm.bias_hh_l0                -> 96 params
lstm.weight_ih_l1              -> 2,304 params
lstm.weight_hh_l1              -> 2,304 params
lstm.bias_ih_l1                -> 96 params
lstm.bias_hh_l1                -> 96 params
fc.weight                      -> 240 params
fc.bias                        -> 10 params


 Total Parameters: 8,794
Estimated Memory: 34.35 KB (0.03 MB)


## Evaluation & Visualization

In [18]:
from src.evaluate import ModelEvaluator

In [19]:
test_instance_2 = ModelEvaluator(
    memory_constraint_model, 
    test_loader,
    device
)

In [20]:
test_instance_2.evaluate()


 Accuracy on test data: 91.33%

 Classification Report:
              precision    recall  f1-score   support

           0     0.9388    0.8846    0.9109        52
           1     0.9062    0.8923    0.8992        65
           2     0.8696    0.9524    0.9091        63
           3     0.9016    0.9167    0.9091        60
           4     1.0000    0.9636    0.9815        55
           5     0.8065    0.9434    0.8696        53
           6     0.8955    0.9231    0.9091        65
           7     0.9107    0.8644    0.8870        59
           8     0.9500    0.9344    0.9421        61
           9     0.9831    0.8657    0.9206        67

    accuracy                         0.9133       600
   macro avg     0.9162    0.9141    0.9138       600
weighted avg     0.9168    0.9133    0.9138       600


 Confusion Matrix:
[[46  0  5  0  0  0  0  1  0  0]
 [ 0 58  2  0  0  2  0  3  0  0]
 [ 2  0 60  1  0  0  0  0  0  0]
 [ 0  0  0 55  0  0  5  0  0  0]
 [ 0  2  0  0 53  0  0  0  0  0]

In [21]:
memory_constraint_model_save_path = os.path.join(project_root_dir, 'outputs', 'models', 'task-b-part-1_weights.pth')
torch.save(memory_constraint_model.state_dict(), memory_constraint_model_save_path)